# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 63 new papers today
          3 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        # highlight authors (FIXME: doc.highlight_authors)
        doc._authors = highlight_authors_in_list(
            [mpia.get_initials(k) for k in doc.authors], 
            normed_mpia_authors, verbose=True)

        full_md = doc.generate_markdown_text()
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/2 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2303.12101


extracting tarball to tmp_2303.12101...

 done.


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Found documentclass in tmp_2303.12101/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:362: LatexWarning: Latex injecting: 'tex/06appendix' from 'tmp_2303.12101/tex/06appendix.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:362: LatexWarning: Latex injecting: 'tex/05conclusion' from 'tmp_2303.12101/tex/05conclusion.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/si

...

/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:120: LatexWarning: attempting recovering figure age_hist_eq_width
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:125: LatexWarning: Recovered figure age_hist_eq_width as tmp_2303.12101/fig/age_hist_eq_width.pdf
  warnings.warn(LatexWarning(f'Recovered figure {image} as {fname}'))


Found 113 bibliographic references in tmp_2303.12101/main.bbl.
Retrieving document from  https://arxiv.org/e-print/2303.12110


extracting tarball to tmp_2303.12110...

 done.


Found 75 bibliographic references in tmp_2303.12110/aanda.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.12101-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.12101) | **Stellar associations powering HII regions $\unicode{x2013}$ I. Defining  an evolutionary sequence**  |
|| F. Scheuermann, et al. -- incl., <mark>K. Kreckel</mark>, <mark>S. Hannon</mark>, <mark>E. Schinnerer</mark> |
|*Appeared on*| *2023-03-23*|
|*Comments*| *15 pages, 12 figures. Accepted for publication in MNRAS*|
|**Abstract**| Connecting the gas in HII regions to the underlying source of the ionizing radiation can help us constrain the physical processes of stellar feedback and how HII regions evolve over time. With PHANGS$\unicode{x2013}$MUSE we detect nearly 24,000 HII regions across 19 galaxies and measure the physical properties of the ionized gas (e.g. metallicity, ionization parameter, density). We use catalogues of multi-scale stellar associations from PHANGS$\unicode{x2013}$HST to obtain constraints on the age of the ionizing sources. We construct a matched catalogue of 4,177 HII regions that are clearly linked to a single ionizing association. A weak anti-correlation is observed between the association ages and the H$\alpha$ equivalent width EW(H$\alpha$), the H$\alpha$/FUV flux ratio and the ionization parameter, log q. As all three are expected to decrease as the stellar population ages, this could indicate that we observe an evolutionary sequence. This interpretation is further supported by correlations between all three properties. Interpreting these as evolutionary tracers, we find younger nebulae to be more attenuated by dust and closer to giant molecular clouds, in line with recent models of feedback-regulated star formation. We also observe strong correlations with the local metallicity variations and all three proposed age tracers, suggestive of star formation preferentially occurring in locations of locally enhanced metallicity. Overall, EW(H$\alpha$) and log q show the most consistent trends and appear to be most reliable tracers for the age of an HII region. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.12110-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.12110) | **The Gas Mass Reservoir of Quiescent Galaxies at Cosmic Noon**  |
|| D. Blánquez-Sesé, et al. -- incl., <mark>E. Schinnerer</mark> |
|*Appeared on*| *2023-03-23*|
|*Comments*| *Accepted to A&A on the 16/03/2023*|
|**Abstract**| We present a 1.1mm stacking analysis of moderately massive (log($M_{*}$/$M_{\odot}$) = 10.7 $\pm$ 0.2) quiescent galaxies (QGs) at $\langle z\rangle \sim1.5$, searching for cold dust continuum emission, an excellent tracer of dust and gas mass. Using both the recent GOODS-ALMA survey as well as the full suite of ALMA Band-6 ancillary data in the GOODS-S field, we report the tentative detection of dust continuum equivalent of dust mass log($M_{dust}$/$M_{\odot}$) = 7.47 $\pm$ 0.13 and gas mass log($M_{gas}$/$M_{\odot}$) = 9.42 $\pm$ 0.14. The emerging gas fraction is $f_{gas}$ = 5.3 $\pm$ 1.8%, consistent with the results of previous stacking analyses based on lower resolution sub(mm) observations. Our results support the scenario where high-z QGs have an order of magnitude larger $f_{gas}$ compared to their local counterparts and have experienced quenching with a non negligible gas reservoir in their interstellar medium - i.e. with gas retention. Subsequent analysis yields an anti-correlation between the $f_{gas}$ and the stellar mass of QGs, especially in the high mass end where galaxies reside in the most massive haloes. The $f_{gas}$ - $M_{*}$ anti-correlation promotes the selection bias as a possible solution to the tension between the stacking results pointing towards gas retention in high-z QGs of moderate $M_{*}$ and the studies of individual targets that favour a fully depleted ISM in massive (log($M_{*}$/$M_{\odot}$) high-z QGs. |

## Failed papers

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2303.12101.md
    + _build/html/tmp_2303.12101/fig/nebulae_age_tracers_corner.png
    + _build/html/tmp_2303.12101/fig/overlap_rgb.png
    + _build/html/tmp_2303.12101/fig/catalogue_properties_2D_hist.png
exported in  _build/html/2303.12110.md
    + _build/html/tmp_2303.12110/./Figures/f_gas_final.png
    + _build/html/tmp_2303.12110/./Figures/fgas_mstar_15.0.png
    + _build/html/tmp_2303.12110/./Figures/Galaxy_selection_2.png
    + _build/html/tmp_2303.12110/./Figures/UVJ_diagram.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\uncertainty}[3]{#1^{+#2}_{-#3}}$
$\newcommand{\StoN}{\mathrm{S}/\mathrm{N}}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\change}[1]{{\color{orange}#1}}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\uncertainty}[3]{#1^{+#2}_{-#3}}$
$\newcommand{\StoN}{\mathrm{S}/\mathrm{N}}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\change}[1]{{\color{orange}#1}}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# Stellar associations powering $\HII$ regions -- I. Defining an evolutionary sequence

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2303.12101-b31b1b.svg)](https://arxiv.org/abs/2303.12101)<mark>Appeared on: 2023-03-23</mark> - _15 pages, 12 figures. Accepted for publication in MNRAS_

</div>
<div id="authors">

F. Scheuermann, et al.

</div>
<div id="abstract">

**Abstract:** Connecting the gas in $\HII$ regions to the underlying source of the ionizing radiation can help us constrain the physical processes of stellar feedback and how $\HII$ regions evolve over time.With PHANGS--MUSE we detect nearly $\num{24000}$ $\HII$ regions across 19 galaxies and measure the physical properties of the ionized gas (e.g. metallicity, ionization parameter, density).We use catalogues of multi-scale stellar associations from PHANGS-- $_HST_$ to obtain constraints on the age of the ionizing sources.We construct a matched catalogue of $\num{4177}$ $\HII$ regions that are clearly linked to a single ionizing association.A weak anti-correlation is observed between the association ages and the $\HA$ equivalent width $\EW$ , the $\HA/\FUV$ flux ratio and the ionization parameter, $\log q$ .As all three are expected to decrease as the stellar population ages, this could indicate that we observe an evolutionary sequence.This interpretation is further supported by correlations between all three properties.Interpreting these as evolutionary tracers, we find younger nebulae to be more attenuated by dust and closer to giant molecular clouds, in line with recent models of feedback-regulated star formation.We also observe strong correlations with the local metallicity variations and all three proposed age tracers, suggestive of star formation preferentially occurring in locations of locally enhanced metallicity.Overall, $\EW$ and $\log q$ show the most consistent trends and appear to be most reliable tracers for the age of an $\HII$ region.

</div>

<div id="div_fig1">

<img src="tmp_2303.12101/fig/nebulae_age_tracers_corner.png" alt="Fig6" width="100%"/>

**Figure 6. -** Comparison between the proposed age tracers for the full \HII region catalogue. The nebulae are grouped by their host galaxy, sorted by stellar mass $M_{\star}$, with the median of the entire sample indicated by a black line. The 68 and 98 percentile ranges are shaded in grey. (*fig:age_tracers_corner*)

</div>
<div id="div_fig2">

<img src="tmp_2303.12101/fig/overlap_rgb.png" alt="Fig1" width="100%"/>

**Figure 1. -** Examples for the overlap between the \HII regions and stellar associations in \galaxyname{NGC}{1365}. The cutouts show a three colour composite images, based on the 5 available _HST_ bands, overlaid with the $\HA$ line emission of MUSE in red. The boundaries of the \HII regions are shown in red and the stellar associations in blue. Flags that characterise the overlap between the two catalogues are showcased in this figure. (*fig:overlap*)

</div>
<div id="div_fig3">

<img src="tmp_2303.12101/fig/catalogue_properties_2D_hist.png" alt="Fig2" width="100%"/>

**Figure 2. -** Distribution of masses and ages of the stellar associations in the matched catalogue (the \texttt{one-to-one} sample). The black lines mark the cuts that we apply to the sample to ensure a fully sampled IMF (more massive than $>\SI{e4}{\Msun}$) and to only include young clusters that should be associated with ionized gas (younger than $\leq\SI{8}{\mega\year}$). The mass cut leaves us with a sample of \num{1014} objects and the age cut with \num{3531}. Applying both cuts results in a sample of \num{756} objects. (*fig:catalogue_properties_2D_hist_v2*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\Mgas}{M_{\rm gas}}$
$\newcommand{\Mdust}{M_{\rm dust}}$
$\newcommand{\Mstar}{M_{\rm \ast}}$
$\newcommand{\Msol}{\rm M_{\rm \odot}}$
$\newcommand{\fgas}{f_{\rm gas}}$
$\newcommand{\fdust}{f_{\rm dust}}$
$\newcommand{\Gobat}{{\color{blue} G18}}$
$\newcommand{\Magdis}{{\color{blue} M21}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\Mgas}{M_{\rm gas}}$
$\newcommand{\Mdust}{M_{\rm dust}}$
$\newcommand{\Mstar}{M_{\rm \ast}}$
$\newcommand{\Msol}{\rm M_{\rm \odot}}$
$\newcommand{\fgas}{f_{\rm gas}}$
$\newcommand{\fdust}{f_{\rm dust}}$
$\newcommand{\Gobat}{{\color{blue} G18}}$
$\newcommand{\Magdis}{{\color{blue} M21}}$</div>



<div id="title">

# The gas mass reservoir of quiescent galaxies at cosmic noon

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2303.12110-b31b1b.svg)](https://arxiv.org/abs/2303.12110)<mark>Appeared on: 2023-03-23</mark> - _Accepted to A&A on the 16/03/2023_

</div>
<div id="authors">

D. Blánquez-Sesé, et al. -- incl., <mark>E. Schinnerer</mark>

</div>
<div id="abstract">

**Abstract:** We present a 1.1mm stacking analysis of moderately massive (log( $\Mstar$ / $\Msol$ ) = 10.7 $\pm$ 0.2) quiescent galaxies (QGs) at $\langle z\rangle \sim1.5$ , searching for cold dust continuum emission, an excellent tracer of dust and gas mass. Using both the recent GOODS-ALMA survey as well as the full suite of ALMA Band-6 ancillary data in the GOODS-S field, we report the tentative detection of dust continuum equivalent of   dust mass log( $\Mdust$ / $\Msol$ ) = 7.47 $\pm$ 0.13 and gas mass log( $\Mgas$ / $\Msol$ ) = 9.42 $\pm$ 0.14. The emerging gas fraction is $\fgas$ = 5.3 $\pm$ 1.8 \% , consistent with the results of previous stacking analyses based on lower resolution sub(mm) observations. Our results support the scenario where high $-z$ QGs have an order of magnitude larger $\fgas$ compared to their local counterparts and have experienced quenching with a non negligible gas reservoir in their interstellar medium - i.e. with gas retention. Subsequent analysis yields an anti-correlation between the $\fgas$ and the stellar mass of QGs, especially in the high mass end where galaxies reside in the most massive haloes. The $\fgas$ - $\Mstar$ anti-correlation promotes the selection bias as a possible solution to the tension between the stacking results pointing towards gas retention in high $-z$ QGs of moderate $\Mstar$ and the studies of individual targets that favour a fully depleted ISM in massive (log( $\Mstar$ / $\Msol$ ) > 11.2) high $-z$ QGs.

</div>

<div id="div_fig1">

<img src="tmp_2303.12110/./Figures/f_gas_final.png" alt="Fig6.1" width="50%"/><img src="tmp_2303.12110/./Figures/fgas_mstar_15.0.png" alt="Fig6.2" width="50%"/>

**Figure 6. -** **Gas and dust fractions of QGs**_Top_: Selection of $\fdust$ and $\fgas$ measurements as a function of  redshift for QGs. Circles correspond to dust derived gas fractions: this work and  previous stacks studies ($\Gobat$ and $\Magdis$) are shown in red and blue, respectively. The white circles show two different estimates (connected by a grey dotted line) for a sample of individually observed lensed galaxies. The lower values correspond to those presented in  ([Whitaker, Williams and Mowla (2021)]())  and the upper values show the new estimates provided in  ([Gobat and Liu (2022)]()) . The grey diamonds represent CO derived $\fgas$ estimates  ([Sargent, Daddi and Bournaud 2015](), [Bezanson, Spilker and Williams 2019](), [Williams, Spilker and Whitaker 2021]()) . The red dashed area embeds $\fgas$ measurements of local QGs obtained for the ATLAS3D sample  ([Young, Bureau and Davis 2011](), [Cappellari, McDermid and Alatalo 2013](), [Davis, Young and Crocker 2014]()) . The blue shaded area and the purple dashed line represent the best fit to the $\Magdis$ data and the  ([Gobat, Magdis and Valentino (2020)]())  model respectively. For reference, we add the $\fgas$ evolution of main sequence galaxies according to  ([Liu, Lang and Magnelli (2019)]()) . _Bottom_: Dust and gas fraction as a function of stellar mass for measurements at $z \sim 1.5$. The symbols are the same as in the top panel. The dotted line shows the $\fgas$ prediction according to the  ([Davé, Finlator and Oppenheimer (2012)]())  galaxy evolution models, color coded as a function of $M_{\mathrm{halo}}$. For reference, we add the $\fgas$ - $\Mstar$ trend measured by  ([Magdis, Daddi and Béthermin (2012)](), [Liu, Lang and Magnelli (2019)]()) . The light yellow scattered diamonds and arrows mark the $\fgas$ detections and upper limits for local QGs with the corresponding best fit plotted as a black dashed line. (*fig:f_gas_plot*)

</div>
<div id="div_fig2">

<img src="tmp_2303.12110/./Figures/Galaxy_selection_2.png" alt="Fig1" width="100%"/>

**Figure 1. -** **Galaxy selection.** Density plot of the used ZFOURGE catalogue in the redshift vs stellar mass plane, consisting of a total 13299 galaxies. The red shaded area represents the region covered by our redshift and stellar mass selection criteria ($1 < z < 3$ and 10.20 $<$ log($\Mstar$/$\Msol$) $<$ 11.50) embedding 852 sources. The orange circles correspond to the QGs that constitute our final selection.  (*fig:galaxy_selection*)

</div>
<div id="div_fig3">

<img src="tmp_2303.12110/./Figures/UVJ_diagram.png" alt="Fig2" width="100%"/>

**Figure 2. -** **UVJ colour diagram.** Distribution of the parent sample of 435 galaxies that meet our selection criteria in the $U-V$, $V-J$ colour-colour space, colour coded by their log(SFR). The red box represents the quiescent region limits defined in  ([Schreiber, Pannella and Elbaz (2015)]()) , which enclose the 140 QGs from which we draw our final sample. (*fig:UVJ_diagram*)

</div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

203  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

6  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
